In [1]:
from weave_example_demo.llm_types.prompts import PromptTemplate
from weave_example_demo.llm_types.rag.vector_store import VectorStore
from weave_example_demo.scorers.llm_guard_scorer import LLMGuardScorer
from weave_example_demo.scorers.tonic_validate_scorer import TonicValidateScorer
from weave_example_demo.llm_types.rag.rag import RAGModel


/Users/anishshah/Documents/GitHub/snowflake-arctic-weave-demo/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
import weave

In [3]:
weave.init('bioasq-rag-data')
vector_store = weave.ref('VectorStore:latest').get()
embedding_model = weave.ref('SentenceTransformersModel:latest').get()

Logged in as Weights & Biases user: a-sh0ts.
View Weave data at https://wandb.ai/a-sh0ts/bioasq-rag-data/weave


In [5]:
embedding_model.model


"SentenceTransformer(\n  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel \n  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})\n  (2): Normalize()\n)"

In [6]:
vector_store.set_embedding_model(embedding_model)

🍩 https://wandb.ai/a-sh0ts/bioasq-rag-data/r/call/2d10f812-5104-4e46-921c-6104dc1c4aa0


In [7]:
_context = vector_store._get_chebyshev_distances("What is MSR", n=2)
_context = vector_store.get_most_relevant_documents("What is MSR", n=2)


/Users/anishshah/Documents/GitHub/snowflake-arctic-weave-demo/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🍩 https://wandb.ai/a-sh0ts/bioasq-rag-data/r/call/be55fabc-264f-4292-b560-d8effb0be0ec


/Users/anishshah/Documents/GitHub/snowflake-arctic-weave-demo/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🍩 https://wandb.ai/a-sh0ts/bioasq-rag-data/r/call/f0b9c4b1-44a1-4470-95e1-8ce2133165c9


In [8]:
_context

[{'document': TraceDict({'passage': ObjectRef(entity='a-sh0ts', project='bioasq-rag-data', name='TextCorpusFiltered', digest='77OxkEhXzsBqaN73ElmxTdZW8iWMhJAOqaG9TqmGT9E', extra=['attr', 'rows', 'id', 'hcSV0sXeRml9nrqnlNkVDwxJygi06pB5xEDF6XVF1IE', 'key', 'passage']), 'id': ObjectRef(entity='a-sh0ts', project='bioasq-rag-data', name='TextCorpusFiltered', digest='77OxkEhXzsBqaN73ElmxTdZW8iWMhJAOqaG9TqmGT9E', extra=['attr', 'rows', 'id', 'hcSV0sXeRml9nrqnlNkVDwxJygi06pB5xEDF6XVF1IE', 'key', 'id'])}),
  'score': 0.24590397466713787},
 {'document': TraceDict({'passage': ObjectRef(entity='a-sh0ts', project='bioasq-rag-data', name='TextCorpusFiltered', digest='77OxkEhXzsBqaN73ElmxTdZW8iWMhJAOqaG9TqmGT9E', extra=['attr', 'rows', 'id', '3vG4sryOI0bbWxXqxGxrUpIOYkRiTxD1DHzb8Cu8dKs', 'key', 'passage']), 'id': ObjectRef(entity='a-sh0ts', project='bioasq-rag-data', name='TextCorpusFiltered', digest='77OxkEhXzsBqaN73ElmxTdZW8iWMhJAOqaG9TqmGT9E', extra=['attr', 'rows', 'id', '3vG4sryOI0bbWxXqxGxrUpIO

In [9]:
system_prompt = """
### Instruction ###
You are an expert medical assistant. Your task is to provide accurate and concise answers to medical questions based on the provided data. Ensure that your responses are clear, informative, unbiased, and avoid relying on stereotypes. Answer in a natural, human-like manner. Here are some examples:

### Example ###
1. Which enzyme deficiency can cause GM1 gangliosidoses?
   - GM1 gangliosidoses are associated with deficiency of β-galactosidase.

2. Which are the drugs utilized for the burning mouth syndrome?
   - Dopaminergic drugs should be given in patients with BMS. Catuama reduces the symptoms of BMS and may be a novel therapeutic strategy for the treatment of this disease. Capsaicin, alpha-lipoic acid (ALA), and clonazepam were those that showed more reduction in symptoms of BMS. Treatment with placebos produced a response that was 72% as large as the response to active drugs.

3. Describe a diet that reduces the chance of kidney stones.
   - People can help prevent kidney stones by making changes in fluid intake and, depending on the type of kidney stone, changes in consumption of sodium, animal protein, calcium, and oxalate. Drinking enough fluids each day is the best way to help prevent most types of kidney stones. Health care providers recommend that a person drink 2 to 3 liters of fluid a day. People with cystine stones may need to drink even more. Though water is best, other fluids may also help prevent kidney stones, such as citrus drinks.

4. Does smoking increase risk for glioblastoma?
   - No. Smoking does not increase risk for glioblastoma.

5. Inhibition of which enzyme is the mechanism of action of alisertib?
   - Alisertib (MLN8237) is a selective Aurora kinase inhibitor that acts by interfering with spindle organization and chromosome alignment during mitosis. It has been tested in patients with gastric cancer, breast cancer, relapsed and refractory aggressive B- and T-cell non-Hodgkin lymphomas, epithelial ovarian, fallopian tube, and primary peritoneal carcinoma.

### Instruction ###
Provide your answers in a similar format. Ensure that your answer is unbiased and avoids relying on stereotypes.
"""

In [10]:
human_prompt = """
### Question ###
{question}

### Contexts ###
{context}

### Answer ###
"""

In [11]:
class BioASQRAGAgent(RAGModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def post_process_result(self, answer: str, context: list) -> dict:
        return {"answer": answer, "context": [doc["document"]["passage"] for doc in context], "all_context": [doc["document"]["passage"] for doc in context]}

In [12]:
rag_model = BioASQRAGAgent(
    vector_store=vector_store,
    system_prompt=system_prompt,
    human_prompt=human_prompt
)


🍩 https://wandb.ai/a-sh0ts/bioasq-rag-data/r/call/b57f0109-d28a-4af5-b6e7-f94506ba4739


In [13]:
rag_model.predict("What is MSR")



/Users/anishshah/Documents/GitHub/snowflake-arctic-weave-demo/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


🍩 https://wandb.ai/a-sh0ts/bioasq-rag-data/r/call/89017272-fdf1-4b5e-adbe-3269fd62293d


{'answer': 'MSR stands for Motor Neurone Disease, which is also known as amyotrophic lateral sclerosis (ALS). It is a serious progressive neurological disorder characterized by the loss of upper motor neurons (UMN) and lower motor neurons (LMN). The exact cause of the disease is still unknown, and there is currently no effective treatment available. Various abnormalities in neurotransmitters, neuropeptides, gene expression, and glutamate metabolism have been observed in patients with ALS. Some theories suggest factors like deficits in motor neuron trophic factors, trans-synaptic degeneration, impaired detoxification abilities, and impaired DNA/RNA metabolism may contribute to the disease. Additionally, genetic factors, such as mutations in the Cu/Zn SOD gene, have been implicated in familial forms of ALS. Therapeutic trials for ALS include agents like CNTF, IGF1, glutamate antagonists, branched-chain amino acids, and TRH analogues.',
 'context': ['Merck KGaA is developing matuzumab, a 

In [14]:
qap = weave.ref('QuestionAnswerPairsTrainFiltered:latest').get()

In [15]:
scorers = [
    TonicValidateScorer(
        metrics=[
            "AnswerSimilarityMetric",
            "AugmentationPrecisionMetric",
            "AnswerConsistencyMetric",
        ]
    ),
    LLMGuardScorer(
        metrics=["NoRefusal", "Relevance", "Sensitive"]),
]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
sub_qap = qap.rows[:10]

In [17]:
evaluation = weave.Evaluation(dataset=sub_qap, scorers=scorers)
await evaluation.evaluate(rag_model)

Retrieving responses: 100%|██████████| 1/1 [00:00<00:00, 14513.16it/s]

Retrieving responses: 100%|██████████| 1/1 [00:00<00:00, 14979.66it/s]



Retrieving responses: 100%|██████████| 1/1 [00:00<00:00, 16384.00it/s]





Retrieving responses: 100%|██████████| 1/1 [00:00<00:00, 14716.86it/s]







Retrieving responses: 100%|██████████| 1/1 [00:00<00:00, 11915.64it/s]









Retrieving responses: 100%|██████████| 1/1 [00:00<00:00, 17189.77it/s]











Retrieving responses: 100%|██████████| 1/1 [00:00<00:00, 19239.93it/s]













Retrieving responses: 100%|██████████| 1/1 [00:00<00:00, 20460.02it/s]















Retrieving responses: 100%|██████████| 1/1 [00:00<00:00, 16710.37it/s]








Scoring responses: 100%|██████████| 1/1 [00:09<00:00,  9.73s/it]


2024-06-15 03:03:43 [debug    ] Initialized classification model device=device(type='mps') model=Model(path='ProtectAI/distilroberta-base-rejection-v1', subfolder='', revision='65584967c3f22ff7723e5370c65e0e76791e6055', onnx_path='ProtectAI/distilroberta-base-rejection-v1', onnx_revision='65584967c3f22ff7723e5370c65e0e76791e6055', onnx_subfolder='onnx', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='mps'), 'return_token_type_ids': False, 'max_length': 128, 'truncation': True}, tokenizer_kwargs={})
2024-06-15 03:03:44 [debug    ] Initialized model              device=device(type='mps') model=Model(path='BAAI/bge-base-en-v1.5', subfolder='', revision='a5beb1e3e68b9ab74eb54cfd186867f64f240e1a', onnx_path='BAAI/bge-base-en-v1.5', onnx_revision='a5beb1e3e68b9ab74eb54cfd186867f64f240e1a', onnx_subfolder='onnx', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='mps')}, tokenizer_kwargs={})
2024-06-1


Scoring responses: 100%|██████████| 1/1 [00:09<00:00,  9.93s/it]


2024-06-15 03:03:45 [debug    ] Initialized NER model          device=device(type='mps') model=Model(path='Isotonic/deberta-v3-base_finetuned_ai4privacy_v2', subfolder='', revision='9ea992753ab2686be4a8f64605ccc7be197ad794', onnx_path='Isotonic/deberta-v3-base_finetuned_ai4privacy_v2', onnx_revision='9ea992753ab2686be4a8f64605ccc7be197ad794', onnx_subfolder='onnx', onnx_filename='model.onnx', kwargs={}, pipeline_kwargs={'batch_size': 1, 'device': device(type='mps'), 'aggregation_strategy': 'simple'}, tokenizer_kwargs={'model_input_names': ['input_ids', 'attention_mask']})
2024-06-15 03:03:45 [debug    ] Loaded regex pattern           group_name=CREDIT_CARD_RE
2024-06-15 03:03:45 [debug    ] Loaded regex pattern           group_name=UUID
2024-06-15 03:03:45 [debug    ] Loaded regex pattern           group_name=EMAIL_ADDRESS_RE
2024-06-15 03:03:45 [debug    ] Loaded regex pattern           group_name=US_SSN_RE
2024-06-15 03:03:45 [debug    ] Loaded regex pattern           group_name=BTC_

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


...

Evaluated 9 of 10 examples

Evaluated 10 of 10 examples

Evaluation summary
{
    'TonicValidateScorer': {
        'answer_similarity': {'mean': 4.3},
        'augmentation_precision': {'mean': 0.4},
        'answer_consistency': {'mean': 0.4642857142857143}
    },
    'LLMGuardScorer': {
        'results_valid': {
            'NoRefusal': {'true_count': 10, 'true_fraction': 1.0},
            'Relevance': {'true_count': 10, 'true_fraction': 1.0},
            'Sensitive': {'true_count': 10, 'true_fraction': 1.0}
        },
        'results_score': {'NoRefusal': {'mean': 0.0}, 'Relevance': {'mean': 0.0}, 'Sensitive': {'mean': 0.0}}
    },
    'model_latency': {'mean': 27.561893129348753}
}

🍩 https://wandb.ai/a-sh0ts/bioasq-rag-data/r/call/b3781423-33bb-4f64-8262-867b48365f12


{'TonicValidateScorer': {'answer_similarity': {'mean': 4.3},
  'augmentation_precision': {'mean': 0.4},
  'answer_consistency': {'mean': 0.4642857142857143}},
 'LLMGuardScorer': {'results_valid': {'NoRefusal': {'true_count': 10,
    'true_fraction': 1.0},
   'Relevance': {'true_count': 10, 'true_fraction': 1.0},
   'Sensitive': {'true_count': 10, 'true_fraction': 1.0}},
  'results_score': {'NoRefusal': {'mean': 0.0},
   'Relevance': {'mean': 0.0},
   'Sensitive': {'mean': 0.0}}},
 'model_latency': {'mean': 27.561893129348753}}

In [18]:
from weave_example_demo.llm_types.agent.tools import RAGTool, ToolRegistry
from weave_example_demo.llm_types.agent.agent import LLMAgentModel

In [19]:
agent_tool_registry = ToolRegistry()


In [20]:
agent_tool_registry.add_tool(
    key="RAGTool",
    function=RAGTool(rag_model=rag_model),
    description="Generate a medical response using the BioASQ RAG model",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The medical question for the BioASQ RAG model",
            },
            "n_documents": {
                "type": "integer",
                "description": "The number of medical documents to retrieve",
                "default": 2,
            },
        },
        "required": ["question"],
    },
)

In [21]:
bioasq_agent_system_prompt = (
    "### Instruction ###\n"
    "You are an intelligent assistant named BioASQ-Bot, an expert in biomedical question answering. Your task is to provide accurate, relevant, and clear answers to biomedical questions using the information from the provided tools and context.\n\n"
    "### Tools ###\n"
    "You have access to the following tools:\n"
    "{tools}\n\n"
    "### Process ###\n"
    "Follow this step-by-step process to answer the question:\n"
    "1. Analyze the question to identify the key information needed.\n"
    "2. Use the available tools to retrieve relevant information.\n"
    "3. Synthesize the retrieved information into a coherent answer.\n"
    "4. Cite the sources used to formulate your answer.\n"
    "5. If the question is unclear or lacks context, ask for clarification.\n"
    "6. Provide the final answer in a clear, unbiased, and professional manner.\n\n"
    "### Format ###\n"
    "Use the following format for your response:\n"
    "Question: <The input question you must answer>\n"
    "Thought: <Your analysis of the question and plan to answer it>\n"
    "Action: <The action to take, should be one of the provided tools>\n"
    "Action Input: <The input to the selected action>\n"
    "Observation: <The result of the action>\n"
    "... (Repeat Thought/Action/Action Input/Observation as needed)\n"
    "Thought: <Your reflection on whether you can now answer the question>\n"
    "FINISH: <The final answer to the original question>\n\n"
    "### Evaluation ###\n"
    "Your performance will be evaluated based on:\n"
    "- Accuracy and relevance of your answers\n"  
    "- Clarity and coherence of your explanations\n"
    "- Proper citation of sources\n"
    "- Ability to ask for clarification when needed\n"
    "- Professionalism and objectivity in your responses\n\n"
    "### Instruction ###\n"
    "You must start your final answer with 'FINISH:'. Do not provide any information that is not supported by the tools and context provided. If you cannot answer the question based on the available information, say 'FINISH: I do not have enough information to answer this question.' Let's begin!\n"
)

bioasq_agent_human_prompt = "Question: {question}\n" "Response:"



In [22]:
allowed_tools = [
         "RAGTool" #BUG: Slice issue
    ]
agent = LLMAgentModel(
    model_name="gpt-3.5-turbo",
    allowed_tools=allowed_tools,
    agent_tool_registry=agent_tool_registry,
    human_prompt=bioasq_agent_human_prompt,
    system_prompt=bioasq_agent_system_prompt,
)



In [23]:
agent.predict("What is MSR? Generate a medical response using the BioASQ RAG model")

03:04:21 - LiteLLM:INFO: utils.py:1298 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://api.openai.com/v1/ \
-d '{'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': "### Instruction ###\nYou are an intelligent assistant named BioASQ-Bot, an expert in biomedical question answering. Your task is to provide accurate, relevant, and clear answers to biomedical questions using the information from the provided tools and context.\n\n### Tools ###\nYou have access to the following tools:\nRAGTool\n\n### Process ###\nFollow this step-by-step process to answer the question:\n1. Analyze the question to identify the key information needed.\n2. Use the available tools to retrieve relevant information.\n3. Synthesize the retrieved information into a coherent answer.\n4. Cite the sources used to formulate your answer.\n5. If the question is unclear or lacks context, ask for clarification.\n6. Provide the final answer in a clear, unbiased, and professional manner.\n\n### Fo

Message(content='Thought: The question is asking about the meaning or definition of "MSR" in a medical context. I will use the BioASQ RAG model to generate a medical response for this question.\n\nAction: RAGTool\nAction Input: { question: "What is MSR?" }\nObservation: \n\nThought: I will now wait for the response generated by the BioASQ RAG model to provide the accurate medical information about MSR.', role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"question":"What is MSR?"}', name='RAGTool'), id='call_Ets49PJ2iKIJ0VgtKJ3jIpCV', type='function')])
True


Batches: 100%|██████████| 1/1 [00:00<00:00, 33.58it/s]
03:04:32 - LiteLLM:INFO: utils.py:1298 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://api.openai.com/v1/ \
-d '{'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '\n### Instruction ###\nYou are an expert medical assistant. Your task is to provide accurate and concise answers to medical questions based on the provided data. Ensure that your responses are clear, informative, unbiased, and avoid relying on stereotypes. Answer in a natural, human-like manner. Here are some examples:\n\n### Example ###\n1. Which enzyme deficiency can cause GM1 gangliosidoses?\n   - GM1 gangliosidoses are associated with deficiency of β-galactosidase.\n\n2. Which are the drugs utilized for the burning mouth syndrome?\n   - Dopaminergic drugs should be given in patients with BMS. Catuama reduces the symptoms of BMS and may be a novel therapeutic strategy for the treatment of this disease. Capsaicin, alpha-lipoic acid (ALA), 

Message(content='FINISH: MSR stands for Matuzumab-Specific Receptor. Matuzumab is a fully humanized epidermal growth factor receptor (EGFR)-specific monoclonal antibody developed by Merck KGaA. It is currently undergoing phase II clinical trials for the treatment of EGFR-bearing tumors such as gastric, cervical, pancreatic, and ovarian cancers.', role='assistant')
False
🍩 https://wandb.ai/a-sh0ts/bioasq-rag-data/r/call/57a47661-b499-49da-a406-bb28700a5812


'FINISH: MSR stands for Matuzumab-Specific Receptor. Matuzumab is a fully humanized epidermal growth factor receptor (EGFR)-specific monoclonal antibody developed by Merck KGaA. It is currently undergoing phase II clinical trials for the treatment of EGFR-bearing tumors such as gastric, cervical, pancreatic, and ovarian cancers.'

In [ ]:
#TODO: Add agent eval